### Database Schema Changes ###
With the transition from Opsim v3 to Opsim v4, the additional of alternative scheduler output databases, and the evolution of MAF to default to v4 databases --- the time has come to put into play kwargs for metrics, slicers and stackers! There is some variation in the database schema between v3 and v4 and alt schedulers: column names have changed and significantly, all angles in v4 are in degrees while in v3 they were in radians. MAF 2.5.0 and above should be able to handle these variations, but if you are not using a standard v4 output database, you will likely have to do some kwarg configuration. 

### tl;dr ###

Metrics, slicers and stackers which deal with values that could be in degrees or radians should now have a kwarg that is something like "degrees" (or latLonDeg for the HealpixSlicer). The defaults are that these kwargs are set to "True", and angles are then assumed to be in degrees and return values will also be in degrees. If they are set to "False", then angles are assumed to be in radians and return values will also be in radians.

Columns where the names have been changed now default to opsim v4 values; kwargs will let you set them to other values. Typical places to have to set these kwargs are in the OpsimFieldSlicer and various stackers. 

You can check available kwargs and their defaults by doing<br>
```help(slicers.HealpixSlicer)```
or 
```help(stackers.RandomDitherFieldPerNightStacker)```.


#### Why can't MAF find the data in the database? ####
With the schema changes, the default columns required for your metrics, slicers and stackers may not be in the database. So how can you tell what columns your metricBundle (a metric + slicer + stackers) is going to look for? 

Here's some examples! 


In [1]:
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as mb

In [2]:
m = metrics.CountMetric(col='expMJD')
s = slicers.HealpixSlicer(nside=64, lonCol='fieldRA', latCol='fieldDec', latLonDeg=False)
bb = mb.MetricBundle(metric=m, slicer=s)
print("Find the columns that will be requested from the database:", bb.dbCols)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Find the columns that will be requested from the database: {'fieldRA', 'expMJD', 'fieldDec'}


In [3]:
m = metrics.CountMetric(col='expMJD')
s = slicers.HealpixSlicer(nside=64, lonCol='randomDitherFieldPerNightRa', latCol='randomDitherFieldPerNightDec', 
                          latLonDeg=False)
st = stackers.RandomDitherFieldPerNightStacker(fieldIdCol='fieldID', degrees=False)
bb = mb.MetricBundle(metric=m, slicer=s, stackerList=[st])
print("Find the columns that will be requested from the database:", bb.dbCols)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Find the columns that will be requested from the database: {'fieldID', 'expMJD', 'night', 'fieldDec', 'fieldRA'}


In [4]:
m = metrics.CountMetric(col='expMJD')
s = slicers.HealpixSlicer(nside=64, lonCol='randomDitherFieldPerVisitRa', latCol='randomDitherFieldPerVisitDec', 
                          latLonDeg=False)
st = stackers.RandomDitherFieldPerVisitStacker(degrees=False)
bb = mb.MetricBundle(metric=m, slicer=s, stackerList=[st])
print("Find the columns that will be requested from the database:", bb.dbCols)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Find the columns that will be requested from the database: {'expMJD', 'fieldRA', 'fieldDec'}


In [5]:
# Note that a stacker will be run, EVEN if we're not actually using those columns in our metric/slicer. 
# So if you have a stacker in the bundle, those columns will be pulled from the database. 
m = metrics.CountMetric(col='expMJD')
s = slicers.HealpixSlicer(nside=64, lonCol='fieldRA', latCol='fieldDec', latLonDeg=False)
st = stackers.RandomDitherFieldPerNightStacker(fieldIdCol='fieldID', degrees=False)
bb = mb.MetricBundle(metric=m, slicer=s, stackerList=[st])
print("Find the columns that will be requested from the database:", bb.dbCols)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes
Find the columns that will be requested from the database: {'fieldID', 'expMJD', 'night', 'fieldDec', 'fieldRA'}


In [8]:
help(stackers.RandomDitherFieldPerNightStacker)

Help on class RandomDitherFieldPerNightStacker in module lsst.sims.maf.stackers.ditherStackers:

class RandomDitherFieldPerNightStacker(RandomDitherFieldPerVisitStacker)
 |  Randomly dither the RA and Dec pointings up to maxDither degrees from center,
 |  one dither offset per new night of observation of a field.
 |  e.g. visits within the same night, to the same field, have the same offset.
 |  
 |  Parameters
 |  ----------
 |  raCol : str, optional
 |      The name of the RA column in the data.
 |      Default 'fieldRA'.
 |  decCol : str, optional
 |      The name of the Dec column in the data.
 |      Default 'fieldDec'.
 |  degrees : bool, optional
 |      Flag whether RA/Dec should be treated as (and kept as) degrees.
 |  fieldIdCol : str, optional
 |      The name of the fieldId column in the data.
 |      Used to identify fields which should be identified as the 'same'.
 |      Default 'fieldId'.
 |  nightCol : str, optional
 |      The name of the night column in the data.
 | 